# Create meeting minutes from an Audio file
For this project, the UI allows you to either upload meeting minutes, or record something of your own!

In [ ]:
# --- Install deps ---
!pip install -q gradio torch==2.5.1+cu124 torchvision==0.20.1+cu124 torchaudio==2.5.1+cu124 --index-url https://download.pytorch.org/whl/cu124
!pip install -q requests bitsandbytes==0.46.0 transformers==4.48.3 accelerate==1.3.0 openai

In [ ]:
# --- Imports ---
import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
from openai import OpenAI
from huggingface_hub import login
from google.colab import userdata
from google.colab import drive
import os

In [ ]:
# --- Constants ---
AUDIO_MODEL = "whisper-1"
LLAMA = "meta-llama/Meta-Llama-3.1-8B-Instruct"

In [ ]:
# --- Auth ---
# assumes Colab userdata or your own env vars
hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

In [ ]:
openai_api_key = userdata.get('OPENAI_API_KEY')
openai = OpenAI(api_key=openai_api_key)

In [ ]:
# --- Model setup ---
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

tokenizer = AutoTokenizer.from_pretrained(LLAMA)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(
    LLAMA, device_map="auto", quantization_config=quant_config
)

In [ ]:
# --- Processing function ---
def process_meeting(audio_file):
    # Step 1: Transcribe
    with open(audio_file, "rb") as f:
        transcription = openai.audio.transcriptions.create(
            model=AUDIO_MODEL, file=f, response_format="text"
        )

    # Step 2: Prepare prompt
    system_message = (
        "You are an assistant that produces minutes of meetings from transcripts, "
        "with summary, key discussion points, takeaways and action items with owners, "
        "in markdown."
    )
    user_prompt = (
        f"Below is an extract transcript of a meeting. Please write minutes in markdown, "
        f"including a summary with attendees, location and date; discussion points; "
        f"takeaways; and action items with owners.\n{transcription}"
    )
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt}
    ]

    # Step 3: Run through LLaMA
    inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
    outputs = model.generate(inputs, max_new_tokens=2000)

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# --- Gradio UI ---
with gr.Blocks() as demo:
    gr.Markdown("## 📝 Meeting Minutes Generator\nUpload an audio file and get structured meeting minutes.")
    with gr.Row():
        audio_in = gr.Audio(type="filepath", label="Upload Meeting Audio")
    btn = gr.Button("Generate Minutes")
    md_out = gr.Markdown()

    btn.click(fn=process_meeting, inputs=audio_in, outputs=md_out)

In [ ]:
demo.launch()